# Informer Paddle Demo

### 安装依赖包
- `poetry install` 
- 或者用pip安装相应包

In [41]:
import paddle
import pandas as pd
import numpy as np
from utils.tools import dotdict
from exp.exp_informer import Exp_Informer

### 打印基本信息

In [42]:
print(f'numpy version: {np.__version__}')
print(f'paddle version: {paddle.__version__}')
print(f'pandas version: {pd.__version__}')
print(f'device: {paddle.CUDAPlace(0)}')
if paddle.is_compiled_with_cuda():
    gpu_count = paddle.device.cuda.device_count()
    print(f"可用的GPU数量: {gpu_count}")
else:
    print("GPU不可用,将使用CPU")

numpy version: 1.26.4
paddle version: 3.0.0-beta1
pandas version: 2.2.2
device: Place(gpu:0)
可用的GPU数量: 1


# 训练
### 参数

In [43]:
args = dotdict()
args.model = 'informer'
args.data = 'WTH_small'
args.root_path = './data/ETT/'
args.data_path = 'WTH_small.csv'
args.features = 'M'
args.target = 'OT'
args.freq = 'h'
args.checkpoints = './checkpoints/'
args.seq_len = 96
args.label_len = 48
args.pred_len = 24
# ─── ⋆⋅☆⋅⋆ ──
args.enc_in = 12
args.dec_in = 12
args.c_out = 12
args.d_model = 512
args.n_heads = 8
args.e_layers = 2
args.d_layers = 1
args.s_layers = '3,2,1'
args.d_ff = 2048
args.factor = 5
args.padding = 0
args.distil = True
args.dropout = 0.05
args.attn = 'prob'
args.embed = 'timeF'
args.activation = 'gelu'
args.output_attention = True
args.mix = True
args.num_workers = 0
args.itr = 2
args.train_epochs = 6
args.batch_size = 32
args.patience = 4  # early stopping patience
args.learning_rate = 0.0001
args.des = 'test'
args.loss = 'mse'
args.lradj = 'type1'
args.use_amp = False
args.inverse = False
args.use_gpu = True if paddle.device.is_compiled_with_cuda() else False
args.gpu = 0
args.use_multi_gpu = False
args.devices = '0,1,2,3'

In [44]:
if args.use_gpu and args.use_multi_gpu:  # 都gpu训练
    args.devices = args.devices.replace(' ', '')
    device_ids = args.devices.split(',')
    args.device_ids = [int(id_) for id_ in device_ids]
    args.gpu = args.device_ids[0]

data_parser = {
    'WTH_small': {
        'data': 'WTH_small.csv',
        'T': 'WetBulbCelsius',
        'M': [12, 12, 12],
        'S': [1, 1, 1],
        'MS': [12, 12, 1],
        'itr': 1,
        'train_epochs': 3,
        'do_predict': True},
}

if args.data in data_parser.keys():
    data_info = data_parser[args.data]
    args.data_path = data_info['data']
    args.target = data_info['T']
    args.enc_in, args.dec_in, args.c_out = data_info[args.features]

args.detail_freq = args.freq
args.freq = args.freq[-1:]

print('Args in experiment:')
print(args)

Args in experiment:
{'model': 'informer', 'data': 'WTH_small', 'root_path': './data/ETT/', 'data_path': 'WTH_small.csv', 'features': 'M', 'target': 'WetBulbCelsius', 'freq': 'h', 'checkpoints': './checkpoints/', 'seq_len': 96, 'label_len': 48, 'pred_len': 24, 'enc_in': 12, 'dec_in': 12, 'c_out': 12, 'd_model': 512, 'n_heads': 8, 'e_layers': 2, 'd_layers': 1, 's_layers': '3,2,1', 'd_ff': 2048, 'factor': 5, 'padding': 0, 'distil': True, 'dropout': 0.05, 'attn': 'prob', 'embed': 'timeF', 'activation': 'gelu', 'output_attention': True, 'mix': True, 'num_workers': 0, 'itr': 2, 'train_epochs': 6, 'batch_size': 32, 'patience': 4, 'learning_rate': 0.0001, 'des': 'test', 'loss': 'mse', 'lradj': 'type1', 'use_amp': False, 'inverse': False, 'use_gpu': True, 'gpu': 0, 'use_multi_gpu': False, 'devices': '0,1,2,3', 'detail_freq': 'h'}


### 训练

In [48]:
import sys

if not 'informer-paddle' in sys.path:
    sys.path += ['informer-paddle']
print(sys.path)

['C:\\github\\txnj\\informer-paddle\\.venv\\lib\\site-packages\\paddle\\base\\..\\libs', 'C:\\Program Files\\JetBrains\\PyCharm 2024.1.4\\plugins\\python\\helpers-pro\\jupyter_debug', 'C:\\Program Files\\JetBrains\\PyCharm 2024.1.4\\plugins\\python\\helpers\\pydev', 'C:\\github\\txnj\\informer-paddle', 'C:\\Users\\hjs\\AppData\\Local\\Programs\\Python\\Python310\\python310.zip', 'C:\\Users\\hjs\\AppData\\Local\\Programs\\Python\\Python310\\DLLs', 'C:\\Users\\hjs\\AppData\\Local\\Programs\\Python\\Python310\\lib', 'C:\\Users\\hjs\\AppData\\Local\\Programs\\Python\\Python310', 'C:\\github\\txnj\\informer-paddle\\.venv', '', 'C:\\github\\txnj\\informer-paddle\\.venv\\lib\\site-packages', 'C:\\github\\txnj\\informer-paddle\\.venv\\lib\\site-packages\\win32', 'C:\\github\\txnj\\informer-paddle\\.venv\\lib\\site-packages\\win32\\lib', 'C:\\github\\txnj\\informer-paddle\\.venv\\lib\\site-packages\\Pythonwin', 'C:\\github\\txnj\\informer-paddle\\.venv\\lib\\site-packages\\setuptools\\_vendor',

In [45]:
Exp = Exp_Informer
for ii in range(args.itr):
    # setting record of experiments
    setting = '{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_at{}_fc{}_eb{}_dt{}_mx{}_{}_{}'.format(
        args.model,
        args.data,
        args.features,
        args.seq_len,
        args.label_len,
        args.pred_len,
        args.d_model,
        args.n_heads,
        args.e_layers,
        args.d_layers,
        args.d_ff,
        args.attn,
        args.factor,
        args.embed,
        args.distil,
        args.mix,
        args.des, ii)

    exp = Exp(args)  # set experiments
    print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
    exp.train(setting)

    print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
    exp.test(setting)

    if args.do_predict:
        print('>>>>>>>predicting : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
        exp.predict(setting, True)

    paddle.device.cuda.empty_cache()

🖥️Use GPU: cuda:0


RuntimeError: (PreconditionNotMet) The third-party dynamic library (cudnn64_8.dll) that Paddle depends on is not configured correctly. (error code is 126)
  Suggestions:
  1. Check if the third-party dynamic library (e.g. CUDA, CUDNN) is installed correctly and its version is matched with paddlepaddle you installed.
  2. Configure third-party dynamic library environment variables as follows:
  - Linux: set LD_LIBRARY_PATH by `export LD_LIBRARY_PATH=...`
  - Windows: set PATH by `set PATH=XXX;%PATH%`
  - Mac: set  DYLD_LIBRARY_PATH by `export DYLD_LIBRARY_PATH=...` [Note: After Mac OS 10.11, using the DYLD_LIBRARY_PATH is impossible unless System Integrity Protection (SIP) is disabled.] (at ..\paddle\phi\backends\dynload\dynamic_loader.cc:340)
